In [13]:
import csv
import pandas as pd
import numpy as np

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [14]:
clientID = <>
clientSecret = <>

credentials = SpotifyClientCredentials(client_id= clientID,client_secret=clientSecret)
spotify = spotipy.Spotify(client_credentials_manager=credentials)

In [15]:
import spotipy.util as util


username = <>

scope = 'playlist-modify-public'
token = util.prompt_for_user_token(username,client_id=clientID,client_secret=clientSecret,
                                   scope=scope, redirect_uri='http://localhost:8888/callback')

In [112]:
len(set(['a','b','c'])&set(['a','b','d']))

2

In [155]:
class recommender():
    '''Create a set of recommendations'''
    
    def __init__(self,username,scope='playlist-modify-public',token=None,clientID=None,clientSecret=None,credentials=None):
        '''clientID : client ID for API app
        clientSecret: client secret for API app
        token: token for authentification
        scope: scope of endpoint requirement'''
        import pickle
        import pandas as pd
        import numpy as np
        import random
        
        self.username = username
        self.clientID = clientID
        self.clientSecret = clientSecret
        
        ## Scope
        self.scope = scope
        
        ## Credentials
        if self.clientID == None and self.clientSecret == None:
            self.credentials = credentials
        elif (self.clientID==None or self.clientSecret==None) and credentials==None:
            print ('Error')
            raise
        else:
            self.credentials = SpotifyClientCredentials(client_id= clientID,client_secret=clientSecret)
        
        ## Token
        if token == None:
            self.token = util.prompt_for_user_token(self.username,client_id=self.clientID,client_secret=self.clientSecret,
                                   scope=self.scope, redirect_uri='http://localhost:8888/callback')
        else:
            self.token = token
            
            
    def create_user_instance(self):
        '''Creates an authenticated user instance'''
        
        self.user = spotipy.Spotify(auth=self.token,client_credentials_manager=self.credentials)
        return self.user
    
    def user_create_playlist(self,name):
        '''Creates a new playlist for user
        name: name for playlist'''
        
        self.name = name
        self.created_playlist = self.user.user_playlist_create(self.username,name=self.name)
        self.playlist_id = self.created_playlist['id']
        return self.created_playlist
    
    def user_get_playlist_tracks(self, playlist_id = None):
        if playlist_id == None:
            playlist_id = self.playlist_id
        playlist = self.user.user_playlist(self.username,playlist_id=playlist_id)
        track_list = []
        for track in playlist['tracks']['items']:
            track_list.append(track['track']['uri'])
        self.track_list = track_list
        return self.track_list
    
    def user_get_playlist_track_audio_feat(self):
        song_audio_features = dict()
        for i,song in enumerate(self.track_list):
            print i, song
        
            ## Access spotify api to retrieve audio features for specific track
            local_feature = self.user.audio_features(str(song))[0]
            
            to_del = ['id','track_href','uri','analysis_url','type']
            for item in to_del:
                del(local_feature[item])
            
            song_audio_features[song] = local_feature
            track_info = spotify.track(song) 
            song_audio_features[song]['artist_genres'] = spotify.artist(track_info['artists'][0]['uri'])['genres']
            song_audio_features[song]['artist_popularity'] = spotify.artist(track_info['artists'][0]['uri'])['popularity']
            song_audio_features[song]['explicit'] = track_info['explicit']
            
        self.song_audio_features= song_audio_features
        return self.song_audio_features
    
      
    def user_playlist_recommend_tracks(self):
        '''Uses the PCAed audio features of the clustered songs'''
        import pickle
        import random
        from sklearn.preprocessing import StandardScaler
        from sklearn.metrics.pairwise import cosine_similarity
        
        
        df = pd.DataFrame(self.song_audio_features).T
        
        ## shifting following columns to end
        df_side = df[['artist_popularity','explicit','artist_genres']]
        df.drop(labels=['artist_popularity','explicit','artist_genres'],inplace=True,axis=1)
        df = df.merge(df_side,left_index=True,right_index=True)
        df['key'] = df['key'].astype('category')
        df['time_signature'] = df['time_signature'].astype('category')
        
        numeric_feature = pickle.load(open('./new_feature_to_scale.sav','rb'))
        for col in df.columns:
            if col in numeric_feature:
                df[col] = pd.to_numeric(df[col],errors ='coerce')
        
        ## Standard Scaler
        ss = pickle.load(open('./new_scaler.sav','rb'))
        
        df[['duration_ms','loudness','tempo']] = ss.transform(df[['duration_ms','loudness','tempo']])
        
        ## Applying multiplier
        df[['energy','valence']] = df[['energy','valence']].apply(lambda x: (x*1.5)**2)
        df[['tempo']] = df[['tempo']].apply(lambda x: (x*1.2)**2)
        
              
        ## Songs database
        database_df = pickle.load(open('./new_audio_features.sav','rb'))
        
        ## Applying multiplier to database
        database_df[['energy','valence']] = database_df[['energy','valence']].apply(lambda x: (x*1.5)**2) 
        database_df[['tempo']] = database_df[['tempo']].apply(lambda x: (x*1.2)**2)
        
        ## Recommend tracks
        recommended_tracks = []
        
        for i, track in df.iterrows():   ## df is the current playlist to recommend
            
            track_dist = []
            genre_tracks = []
            
            if len(track['artist_genres']) >=5:
                for db_i,db_track in database_df.iterrows():
                    try:
                        ## Append if 3 genres or more intersect
                        if len(set(db_track['artist_genres']) & set(track['artist_genres']))>=3:
                            genre_tracks.append(db_i)
                    except:
                        continue

            else:
                ## For niche artist
                for db_i,db_track in database_df.iterrows():
                    try:
                        if bool(set(db_track['artist_genres']) & set(track['artist_genres'])):
                            genre_tracks.append(db_i)
                    except:
                        continue
            
            ## Computing cosine similarity for each track within the same genre
            for i2, cluster_track in database_df[database_df.index.isin(genre_tracks)].iloc[:,:-3].iterrows():
                cos_dist = cosine_similarity(track[:-3].reshape(1, -1),cluster_track.reshape(1, -1))
                track_dist.append((cos_dist,i2))
            
            try:
                close_position=1
                similar_track = sorted(track_dist,reverse=True)[close_position][1] 
                
                ## Checks if recommended song is already inside playlist
                
                while similar_track in self.track_list or similar_track in recommended_tracks:
                    closeness_position+=1
                    similar_track = sorted(track_dist,reverse=True)[close_position][1]
                    ## Reverse = True for cos sim, if not False
                
                recommended_tracks.append(similar_track) 
                
            except:
                pass
        
        self.recommended_tracks = recommended_tracks
        
        return self.recommended_tracks
    
    
    def user_playlist_add_tracks(self,playlist_id=None,tracks=None,num_tracks_add='max'):
        if playlist_id==None:
            playlist_id = self.playlist_id
        if tracks == None:
            if num_tracks_add == 'max':
                tracks = self.recommended_tracks
            else:
                tracks = []
                for num in range(num_tracks_add):
                    tracks.append(self.recommended_tracks.pop(num))
            
        self.user.user_playlist_add_tracks(username,playlist_id=playlist_id,tracks=tracks)
        print 'tracks added'

-------------

In [145]:
def auto_rec_add(pl_uri,add_tracks='max'):
    uri = pl_uri.split(':')[-1]
    
    test_class = recommender(username=username,scope=scope,clientID=clientID,clientSecret=clientSecret,credentials=credentials)
    print ('Token Retrieved')
    test_class.create_user_instance()
    print ('User Instance Created')
    test_class.user_get_playlist_tracks(uri)
    print ('Playlist Tracks Retrieved')
    test_class.user_get_playlist_track_audio_feat()
    print ('Track features Retrieved')
    test_class.user_playlist_recommend_tracks()
    print ('Tracks recommended')
    test_class.user_playlist_add_tracks(playlist_id=uri,num_tracks_add=add_tracks)

In [156]:
auto_rec_add('spotify:user:xxxxx:playlist:xxxxxx')

User Instance Created
Playlist Tracks Retrieved
0 spotify:track:1aRtduEHwvk2AMR87ji9Tg
1 spotify:track:63yesoRJgXT5QALryYFV0X
2 spotify:track:59wlTaYOL5tDUgXnbBQ3my
3 spotify:track:3VXrpkM94UBgL4voR20tZq
Track features Retrieved


/home/ec2-user/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:152: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


Tracks recommended
tracks added
